# Missing Values: Stats First, Then Optional Imputation

This notebook is structured to show missing-value statistics before any modification. Imputation steps are placed later and gated by a flag so you can review the stats first.


In [ ]:
# Imports and configuration
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

DATA_PATH = 'dynamic_supply_chain_logistics_dataset.csv'
assert os.path.exists(DATA_PATH), f"CSV not found at {DATA_PATH}"

na_tokens = ['', ' ', 'na', 'n/a', 'NA', 'N/A', 'null', 'NULL', 'none', 'None', 'nan', 'NaN', 'NAN', '-', '--']

df = pd.read_csv(DATA_PATH, na_values=na_tokens, keep_default_na=True, low_memory=False)

print('Working directory:', os.getcwd())
print('File size (MB):', round(os.path.getsize(DATA_PATH) / (1024*1024), 2))
print('Rows, Cols:', df.shape)
df.head(3)


In [ ]:
# Missing value statistics (no changes performed)
na_counts = df.isna().sum()
na_pct = (na_counts / len(df) * 100).round(3)

missing_summary = (
    pd.DataFrame({
        'dtype': df.dtypes.astype(str),
        'missing_count': na_counts,
        'missing_pct': na_pct
    })
    .sort_values('missing_pct', ascending=False)
)

missing_summary


In [ ]:
# Visual overview of missingness (still no changes)
plt.figure(figsize=(10, 5))
(missing_summary['missing_pct']
 .sort_values(ascending=False)
 .plot(kind='bar', color='#4C78A8'))
plt.ylabel('Missing %')
plt.title('Missing values percentage by column')
plt.xticks(rotation=75, ha='right')
plt.tight_layout()
plt.show()

# Heatmap on sample to show patterns
sample = df.sample(min(2000, len(df)), random_state=42)
plt.figure(figsize=(12, 6))
sns.heatmap(sample.isna(), cbar=False)
plt.title('Missingness heatmap (sample)')
plt.xlabel('Columns')
plt.ylabel('Sample Rows')
plt.tight_layout()
plt.show()


## Optional: Imputation (Run only after reviewing stats)

The following cells implement imputation strategies:
- Time-series numeric interpolation + forward/backward fill
- Categorical mode fill
- Domain rule for `handling_equipment_availability`
- Optional KNN-based numeric imputation

Nothing will be modified unless you explicitly run the imputation cell and set the flag.


In [ ]:
APPLY_IMPUTATION = False  # set to True to perform imputation

if APPLY_IMPUTATION:
    _df = df.copy()
    _df['timestamp'] = pd.to_datetime(_df['timestamp'], errors='coerce')
    _df = _df.sort_values('timestamp').set_index('timestamp')

    numeric_cols = _df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = [c for c in _df.columns if c not in numeric_cols]

    # Numeric: interpolate then edge fill
    _num = _df[numeric_cols].copy().interpolate(method='time', limit_direction='both').ffill().bfill()

    # Categorical: mode
    _cat = _df[cat_cols].copy()
    for col in _cat.columns:
        if _cat[col].isna().any():
            mode_val = _cat[col].mode(dropna=True)
            _cat[col] = _cat[col].fillna(mode_val.iloc[0] if len(mode_val) else 'UNKNOWN')

    imputed_df = pd.concat([_num, _cat], axis=1).sort_index()[_df.columns]

    # Domain rule: handling_equipment_availability
    col_avail = 'handling_equipment_availability'
    col_handling = 'loading_unloading_time'
    if col_avail in imputed_df and col_handling in imputed_df:
        delay_threshold = _df[col_handling].quantile(0.75)
        to_fill_idx = imputed_df[col_avail].isna() & (_df[col_handling] >= delay_threshold)
        imputed_df.loc[to_fill_idx, col_avail] = 0.0

    # Optional KNN
    USE_KNN = False
    if USE_KNN:
        from sklearn.impute import KNNImputer
        knn = KNNImputer(n_neighbors=5, weights='distance')
        imputed_df[numeric_cols] = knn.fit_transform(imputed_df[numeric_cols])

    # Report and save
    before = df.isna().sum()
    after = imputed_df.reset_index().isna().sum()
    report = pd.DataFrame({'before_missing': before, 'after_missing': after.reindex(before.index, fill_value=0)})
    report['delta'] = report['before_missing'] - report['after_missing']
    display(report.sort_values('delta', ascending=False))

    OUTPUT_PATH = 'dynamic_supply_chain_logistics_dataset_imputed.csv'
    imputed_df.reset_index().to_csv(OUTPUT_PATH, index=False)
    print('Saved imputed dataset to', OUTPUT_PATH)
else:
    print('Imputation not applied. Set APPLY_IMPUTATION=True and re-run this cell after reviewing stats.')
